# 컬리 뷰티 카테고리 상품 크롤러

컬리 API를 사용하여 뷰티 카테고리의 상품 정보를 수집하는 노트북입니다.

## 1. 라이브러리 임포트

In [ ]:
import requests
import json
import pandas as pd
from typing import Dict, List

## 2. API 설정

In [ ]:
# API 엔드포인트 설정
BASE_URL = 'https://api.kurly.com/collection/v2/home/sites/beauty/product-categories'
CATEGORY_ID = 167001  # 뷰티 카테고리 ID

# 요청 파라미터
params = {
    'sort_type': 4,      # 정렬 타입
    'page': 1,           # 페이지 번호
    'per_page': 100,     # 페이지당 상품 수
    'filters': ''        # 필터
}

# URL 생성
url = f'{BASE_URL}/{CATEGORY_ID}/products'
print(f'API URL: {url}')

## 3. API 요청

In [ ]:
# API 요청 실행
response = requests.get(url, params=params)

# 응답 상태 확인
print(f'상태 코드: {response.status_code}')
print(f'응답 크기: {len(response.content)} bytes')

## 4. 데이터 파싱

In [ ]:
# JSON 파싱
data = response.json()

# 응답 구조 확인
print(f'응답 코드: {data["code"]}')
print(f'응답 메시지: {data["message"]}')
print(f'상품 수: {len(data["data"])}개')

In [ ]:
# 상품 정보 추출
products = data['data']

# 첫 번째 상품 정보 확인
sample_product = products[0]
print('샘플 상품 정보:')
print(json.dumps(sample_product, indent=2, ensure_ascii=False)[:500])

## 5. 데이터 가공

In [ ]:
# 필요한 필드만 추출하여 정리
processed_products = []

for product in products:
    processed = {
        '상품번호': product.get('no'),
        '상품명': product.get('name'),
        '설명': product.get('short_description'),
        '정가': product.get('sales_price'),
        '할인가': product.get('discounted_price'),
        '할인율': product.get('discount_rate'),
        '리뷰수': product.get('review_count'),
        '품절여부': product.get('is_sold_out'),
        '이미지URL': product.get('list_image_url')
    }
    processed_products.append(processed)

print(f'가공 완료: {len(processed_products)}개 상품')

## 6. DataFrame 변환

In [ ]:
# DataFrame 생성
df = pd.DataFrame(processed_products)

# 데이터 확인
print(f'DataFrame 크기: {df.shape}')
df.head()

In [ ]:
# 기본 통계
print('=== 가격 통계 ===')
print(df[['정가', '할인가', '할인율']].describe())

print('\n=== 품절 현황 ===')
print(df['품절여부'].value_counts())

## 7. 데이터 저장

In [ ]:
# CSV 저장
df.to_csv('kurly_beauty_products.csv', index=False, encoding='utf-8-sig')
print('CSV 저장 완료: kurly_beauty_products.csv')

# JSON 저장
with open('kurly_beauty_products.json', 'w', encoding='utf-8') as f:
    json.dump(processed_products, f, ensure_ascii=False, indent=2)
print('JSON 저장 완료: kurly_beauty_products.json')

## 8. 페이지네이션 정보

In [ ]:
# 페이지네이션 정보 확인
pagination = data['meta']['pagination']

print('=== 페이지네이션 정보 ===')
print(f'전체 상품 수: {pagination["total"]}개')
print(f'현재 페이지: {pagination["current_page"]}/{pagination["total_pages"]}')
print(f'페이지당 상품 수: {pagination["per_page"]}개')

## 9. 샘플 데이터 확인

In [ ]:
# 할인율 높은 상위 5개 상품
print('=== 할인율 TOP 5 ===')
top_discounted = df.nlargest(5, '할인율')[['상품명', '정가', '할인가', '할인율']]
print(top_discounted.to_string())